# Adaption to new classes
project 7 by Guanzhao Wang,Haochen Wu, Yukai Wang

### Abstact outline:

### Teammates:
- Guanzhao Wang:

- Haochen Wu:

- Yukai Wang:

In [1]:
from mylibs.dataloader import getDataLoaders
from mylibs.train import train
from mylibs.eval import validate
from mylibs.model import Net
from mylibs.loss import loss_function
from mylibs.report import report_summary

import torch
from torchvision import transforms

In [2]:
USE_GPU = False
BATCH_SIZE = 32
EPOCH = 5
NUM_WORKERS = 2
K = 7

In [3]:
device = torch.device("cuda" if USE_GPU else "cpu")

transform = transforms.Compose([
    transforms.ToTensor()
])
train_data_loader, test_data_loader = getDataLoaders(include_labels=range(K), transform=transform, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, mode=1)

Labeling unlabeled data...
[[5710 1025  837]
 [   1 4963    6]
 [ 289   12 5157]]
[[5710 1025  837]
 [   1 4963    6]
 [ 289   12 5157]]
18000
Labeling accuracy: 0.8794444444444445


In [ ]:
model = Net().to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.01)
eval_metrics = []

# scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, EPOCH+1):
    eval_metric = train(train_data_loader, model, loss_function, optimizer, USE_GPU)
    eval_metrics.append(eval_metric)
    print("Epoch: {} Loss: {}".format(epoch, eval_metric["loss"]))

NameError: name 'Net' is not defined

In [ ]:
report_summary(eval_metrics)

In [ ]:
model.eval()
eval_metric = validate(test_data_loader, model, loss_function, USE_GPU)
print("Eval_metrics over test dataset: {}".format(eval_metric))

Loss over test dataset: 0.45216789841651917
